# **Computational Theory Assessment G00417529**

# **Imports**

In [391]:
# numpy and unittest imports
import numpy as np
import unittest
# 32-bit unsigned integer alies to reduce verbosity of notebook
u32 = np.uint32
# Decorator for decorators, keeps function metadata intact
from functools import wraps
# Used for benchmarking function performance
import timeit


## **Introduction**
This section defines technical terms used for all the solutions in this notebook

### **Bitwise Terms and Operations**
**Bitwise Operations** are actions directly applied to individual bits of binary numbers. <br>
**Binary Words** are the natural unit of data a computer CPU can process with fixed lengths like 32bits (4 bytes) or 64 bits (8 bytes) <br><br>

**Bitwise AND ($\land$)/(`&`)**<br>
Compares each bit of two **binary words**.

For every bit position:
- The result bit is `1` only if both corresponding bits in the operands are `1`.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X & Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
</table><br>

**Bitwise OR (`|`)**<br>
For every bit position:
- The bit result is `1` if either bit is `1`.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X | Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
</table><br>

**Bitwise XOR ($\oplus$)/(`^`)**<br>
For every bit position:
- The bit result is `1` if both bits are different.
- Otherwise, the result bit is `0`.

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">Y</th>
    <th style="border: 1px solid black;">X ^ Y</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
</table><br>

**Bitwise NOT (`~`)**<br>
Flips every bit (`0→1`, `1→0`)

<table style="border: 1px solid black; border-collapse: collapse; text-align: center; padding: 8px;">
  <tr>
    <th style="border: 1px solid black;">X</th>
    <th style="border: 1px solid black;">~X</th>
  </tr>
  <tr>
    <td style="border: 1px solid black;">0</td>
    <td style="border: 1px solid black;">1</td>
  </tr>
  <tr>
    <td style="border: 1px solid black;">1</td>
    <td style="border: 1px solid black;">0</td>
  </tr>
</table><br>

## **Helper Functions**
**Used to avoid duplicate code throughout solutions**<br>
**Prevents variable redeclaration**<br>

In [392]:
def run_local_scope_tests(test_class, ver_val=0):
    """Run a unittest class without redeclaring variables in the global scope."""
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=ver_val)
    result = runner.run(suite)
    return result

### Decorator to force all plain int parameters as np.uint32

In [393]:
# IMRPOVE DOCS LATER
def force_u32(func):
    """
    Decorator that automatically converts all integer arguments passed to a function
    into np.uint32 to reduce verbosity and improve convenience.

    We use a decorator as it only has to be tested once, rather than testing
    each individual function for correct argument types.

    It also allows us to improve preformance by avoiding repeated type checks
    within loops while having user faced code that is clean, easy to read and use.
    """

     # Preserve metadata of the original function,
     # such as its name and docstring which is useful for debugging and documentation and allows introspection tools to work correctly.
     # e.g allows tools like help() to work and __name__ to work as expected.
    @wraps(func)
    # Define the wrapper function that will replace the original function
    # *args and **kwargs allow it to accept any number of positional and keyword arguments
    # This makes the decorator flexible and able to wrap functions with different signatures
    # *args collects positional arguments into a tuple
    # **kwargs collects keyword arguments into a dictionary
    def wrapper(*args, **kwargs):
        # using list comprehensions to iterate over all arguments and convert them as needed then storing in tuple
        # Convert positional arguments: if argument is a plain int, cast to np.uint32
        # if it's not a plain int (e.g., already np.uint32 or another type), leave it unchanged
        new_args = tuple(u32(a) if isinstance(a, int) else a for a in args)

        # Convert keyword arguments: same as above for kwargs
        # but we handle key-value pairs in a dictionary
        new_kwargs = {key: u32(value) if isinstance(value, int) else value
                      for key, value in kwargs.items()}

        # Call the original function with converted arguments
        return func(*new_args, **new_kwargs)

    return wrapper  # Return the wrapped function

### force_u32 tests

In [394]:
# --- Internal, fast functions (undecorated) ---
def add(x: u32, y: u32) -> u32:
    """Add two np.uint32 numbers."""
    return x + y

def multiply(x: u32, y: u32) -> u32:
    """Multiply two np.uint32 numbers."""
    return x * y

# --- User-facing, safe but slower functions (decorated) ---
@force_u32
def add_user_facing(x: int, y: int) -> u32:
    """User-facing add function, converts int inputs to np.uint32 automatically."""
    return add(x, y)

@force_u32
def multiply_user_facing(x: int, y: int) -> u32:
    """User-facing multiply function, converts int inputs to np.uint32 automatically."""
    return multiply(x, y)

# --- Unit tests ---
class TestForceU32(unittest.TestCase):

    def test_alising(self):
        """Test that u32 alias works correctly"""
        self.assertEqual(u32(5), np.uint32(5))
        self.assertIsInstance(u32(5), np.uint32)
    
    def test_add_user_facing(self):
        """Test positional args for add_user_facing"""
        result = add_user_facing(7, 3)
        self.assertEqual(result, 10)
        self.assertIsInstance(result, np.uint32)
    
    def test_multiply_user_facing(self):
        """Test positional args for multiply_user_facing"""
        result = multiply_user_facing(3, 7)
        self.assertEqual(result, 21)
        self.assertIsInstance(result, np.uint32)

    def test_keyword_args(self):
        """Test that keyword arguments are converted"""
        result = add_user_facing(x=66, y=11)
        self.assertEqual(result, 77)
        self.assertIsInstance(result, np.uint32)

    def test_mixed_args(self):
        """Test combination of positional and keyword arguments"""
        result = multiply_user_facing(10, y=3)
        self.assertEqual(result, 30)
        self.assertIsInstance(result, np.uint32)

# Run local scope tests
run_local_scope_tests(TestForceU32)

----------------------------------------------------------------------
Ran 5 tests in 0.000s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

### Preformance Demo

In [395]:
def performance_demo():
    """Demonstrate performance difference between decorated and undecorated functions."""
    N = 10**7 # Number of iterations for timing (10^6)
    x, y = 10, 20 # Sample integer inputs

    # Undecorated internal function
    t_internal = timeit.timeit(lambda: add(u32(x), u32(y)), number=N)

    # Decorated user-facing function (with conversion)
    t_user = timeit.timeit(lambda: add_user_facing(x, y), number=N)

    print(f"Internal undecorated function time: {t_internal:.4f} s")
    print(f"User-facing decorated function time: {t_user:.4f} s")

    # Decorator adds overhead for type conversion (x2 slower), but provides safety for user inputs.
    # This trade-off is often worthwhile in user-facing code, we prioritise safety and usability over raw performance.
    # but for inner loops, or methods where type safety is guaranteed, using undecorated functions is preferred.

# Run performance demonstration
# performance_demo()

## **Problem 1: Binary Words and Operations**

### 1.1 Parity Function
$\text{Parity}(x, y, z) = x \oplus y \oplus z$

#### 1.11 Design Decisions
I chose to statically define all parameter types as NumPy 32-bit unsigned integers **(np.uint32)** to avoid writing defensive runtime checks and to improve performance.
In the Secure Hash Standard, this function would be called millions of times per hash computation. While this notebook is primarily meant to demonstrate functionality and could therefore benefit from more flexible, dynamically typed inputs I believe performance is just as critical as correctness in these bitwise operations. After all, these functions are designed to operate at the bit level for efficiency and precision; type checking them with expensive calls would defeat the purpose.

In [396]:
class Parity:
    # interal undecorated function
    @staticmethod
    def __Parity(x: u32, y: u32, z: u32) -> u32:
        """
        Operation:
            Applies bitwise eclusive or (XOR) to three 32-bit unsigned integers (x, y, z).
            For each bit position of x, y and z:
                - if an odd number of bits are 1, the result is 1
                - if an even number of bits are 1, the result is 0

        Parameters:
            (all arguments are of numpy unsigned 32-bit integer type)
            x: First integer
            y: Second integer.
            z: Third integer.

        Returns: 
            result of 'Operation' as numpy unsigned 32-bit integer.
        """

        # Apply bitwise XOR to the three integers and return the result
        return x ^ y ^ z
    
    # user-facing decorated function
    @staticmethod
    @force_u32
    def __Parity_safe(x: int, y: int, z: int) -> u32:
        return Parity.__Parity(x, y, z)
    
    # Unit test method template
    # improves readability and reduces boilerplate code in tests
    @staticmethod
    def __test(test_class: unittest.TestCase, x: int, y: int, z: int, expected: int, fail_msg: str):
        test_class.assertEqual(Parity.__Parity_safe(x, y, z), expected, fail_msg)

    # Expose both versions
    internal = __Parity
    user_facing = __Parity_safe
    assert_me = __test
        

### 1.11 Parity test cases

In [397]:
# Unit tests for the Parity method.
class TestParity(unittest.TestCase):

     def test_base_cases(self):
        """Basic XOR combinations."""
        
        # 0 ⊕ 0 ⊕ 0 = 0
        Parity.assert_me(self, x=0, y=0, z=0, 
                         expected=0, 
                         fail_msg="Failed on all zeros")

        # 1 ⊕ 0 ⊕ 0 = 1
        Parity.assert_me(self, x=1, y=0, z=0, 
                         expected=1, 
                         fail_msg="Failed when only x=1")

        # 1 ⊕ 1 ⊕ 0 = 0
        Parity.assert_me(self, x=1, y=1, z=0, 
                        expected=0, 
                        fail_msg="Failed on even number of 1s")

        # 1 ⊕ 1 ⊕ 1 = 1
        Parity.assert_me(self, x=1, y=1, z=1, 
                expected=1, 
                fail_msg="Failed on odd number of 1s")


run_local_scope_tests(TestParity)

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### 1.2 Ch Function
$\text Ch(x, y, z) = (x \land y) \oplus (\lnot x \land z)$

In [398]:
def Ch(x: u32, y:u32, z:u32) -> u32:
    return (x & y) ^ (~x & z)

In [399]:
class TestCh(unittest.TestCase):

    def test_base_cases(self):
        """Basic Tests for Ch(x, y, z)."""

        # Case 1: x=0, y=0, z=0
        # (0 & 0) ⊕ (~0 & 0) = 0
        x1, y1, z1 = u32(0), u32(0), u32(0)
        expected1 = u32(0)
        self.assertEqual(Ch(x1, y1, z1), expected1, "Failed on all zeros")

        # Case 2: x=0, y=0, z=1
        # (0 & 0) ⊕ (~0 & 1) = 1
        x2, y2, z2 = u32(0), u32(0), u32(1)
        expected2 = u32(1)
        self.assertEqual(Ch(x2, y2, z2), expected2, "Failed when x=0, y=0, z=1")

        # Case 3: x=1, y=0, z=1
        # (1 & 0) ⊕ (~1 & 1) = 0
        x3, y3, z3 = u32(1), u32(0), u32(1)
        expected3 = u32(0)
        self.assertEqual(Ch(x3, y3, z3), expected3, "Failed when x=1, y=0, z=1")

        # Case 4: x=1, y=1, z=0
        # (1 & 1) ⊕ (~1 & 0) = 1
        x4, y4, z4 = u32(1), u32(1), u32(0)
        expected4 = u32(1)
        self.assertEqual(Ch(x4, y4, z4), expected4, "Failed when x=1, y=1, z=0")

        # Case 5: x=1, y=1, z=1
        # (1 & 1) ⊕ (~1 & 1) = 1
        x5, y5, z5 = u32(1), u32(1), u32(1)
        expected5 = u32(1)
        self.assertEqual(Ch(x5, y5, z5), expected5, "Failed when all are 1s")

        # Case 6: x=0, y=1, z=1
        # (0 & 1) ⊕ (~0 & 1) = 1
        x6, y6, z6 = u32(0), u32(1), u32(1)
        expected6 = u32(1)
        self.assertEqual(Ch(x6, y6, z6), expected6, "Failed when x=0, y=1, z=1")

# avoid redeclaring variables in global scope
run_local_scope_tests(TestCh)

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### 1.3 Maj Function
$\text Maj(x, y, z) = (x \land y) \oplus (x \land z) \oplus (y \land z)$

In [400]:
def Maj(x: u32, y: u32, z: u32) -> u32:
    return (x & y) ^ (x & z) ^ (y & z)

#### 1.31 Maj Test Cases

In [401]:
class TestMaj(unittest.TestCase):
    """Unit tests for the Maj(x, y, z) function."""

    def test_base_cases(self):
        """Base tests"""

        # Case 1: x=0, y=0, z=0
        # (0&0) ⊕ (0&0) ⊕ (0&0) = 0
        x1, y1, z1 = u32(0), u32(0), u32(0)
        expected1 = u32(0)
        self.assertEqual(Maj(x1, y1, z1), expected1, "Failed on all zeros")

        # Case 2: x=0, y=0, z=1
        # (0&0) ⊕ (0&1) ⊕ (0&1) = 0
        x2, y2, z2 = u32(0), u32(0), u32(1)
        expected2 = u32(0)
        self.assertEqual(Maj(x2, y2, z2), expected2, "Failed when z=1 only")

        # Case 3: x=0, y=1, z=1
        # (0&1) ⊕ (0&1) ⊕ (1&1) = 1
        x3, y3, z3 = u32(0), u32(1), u32(1)
        expected3 = u32(1)
        self.assertEqual(Maj(x3, y3, z3), expected3, "Failed when x=0, y=1, z=1")

        # Case 4: x=1, y=0, z=1 
        # (1&0) ⊕ (1&1) ⊕ (0&1) = 1
        x4, y4, z4 = u32(1), u32(0), u32(1)
        expected4 = u32(1)
        self.assertEqual(Maj(x4, y4, z4), expected4, "Failed when x=1, y=0, z=1")

        # Case 5: x=1, y=1, z=1
        # (1&1) ⊕ (1&1) ⊕ (1&1) = 1
        x5, y5, z5 = u32(1), u32(1), u32(1)
        expected5 = u32(1)
        self.assertEqual(Maj(x5, y5, z5), expected5, "Failed when all are 1s")

        # SHOW WORKINGS FOR 2 examples, basic and non-standard
        # Case 6: x=7, y=8, z=2
        0b00000111
        0b00001000
        # =
        0b00000000 # P1
        # XOR
        0b00000111
        0b00000010
        # =
        0b00000010 # P2
        # XOR
        0b00001000
        0b00000010
        # =
        0b00000000 # P3

        0b00000000 # P1
        0b00000010 # P2
        0b00000000 # P3
        0b00000010 # Final Result
        # 0b00000010 = 2


        # (7&8) ⊕ (7&2) ⊕ (8&2) = 2
        x6, y6, z6 = u32(7), u32(8), u32(2)
        expected6 = u32(2)
        self.assertEqual(Maj(x6, y6, z6), expected6, "Non-standard case")

# avoid redeclaring variables in global scope
run_local_scope_tests(TestMaj)

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### 1.4 Implementing Sigma functions
#### 1.41 Helper Functions
The sigma functions use the following methods throughout their implementation:
- **ROTR** (Circular right shift Operation)
- **SHR** ( Right Shift Operation )

These functions have been implemented as defined in [FIPS 180-4, Section 3.2: Operations on Words](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as helper methods to reduce repetiton. 

**Circular right shift Operation**<br>
$\text{ROTR}^n(x) = (x \gg n) \lor (x \ll w - n)$

In [402]:
def ROTR(x: u32, n: u32) -> u32:
    """Right rotate a 32-bit word x by n bits."""
    return (x >> n) | (x << (32 - n))

**Right Shift Operation**<br>
$\text{SHR}^n(x) = x \gg n$

In [403]:
def SHR(x: u32, n:u32) -> u32:
    """Logical right shift of x by n bits."""
    return (x >> n)

#### 1.5 large Sigma 0
$\Sigma_{0}^{256}(x) \;=\; \mathrm{ROTR}^2(x) \;\oplus\; \mathrm{ROTR}^{13}(x) \;\oplus\; \mathrm{ROTR}^{22}(x)$

In [404]:
def Sigma0(x: u32) -> u32:
    return ROTR(x, u32(2)) ^ ROTR(x, u32(13)) ^ ROTR(x, u32(22))

#### 1.6 Large Sigma 1
$\Sigma_{1}^{256}(x) \;=\; \mathrm{ROTR}^6(x) \;\oplus\; \mathrm{ROTR}^{11}(x) \;\oplus\; \mathrm{ROTR}^{25}(x)$

In [405]:
def Sigma1(x: u32) -> u32:
    return ROTR(x, u32(6)) ^ ROTR(x, u32(11)) ^ ROTR(x, u32(25))

#### 1.7 Small Sigma 0
$\sigma_{0}^{256}(x) = \mathrm{ROTR}^{7}(x) \;\oplus\; \mathrm{ROTR}^{18}(x) \;\oplus\; \mathrm{SHR}^{3}(x)$

In [406]:
def sigma0(x: u32) -> u32:
    return ROTR(x, u32(7)) ^ ROTR(x, u32(18)) ^ SHR(x, u32(3))

#### 1.7 Small Sigma 1
$\sigma_{1}^{256}(x) = \mathrm{ROTR}^{17}(x) \;\oplus\; \mathrm{ROTR}^{19}(x) \;\oplus\; \mathrm{SHR}^{10}(x)$

In [407]:
def sigma1(x: u32) -> u32:
    return ROTR(x, u32(17)) ^ ROTR(x, u32(19)) ^ SHR(x, u32(10))

## Problem 2: Fractional Parts of Cube Roots

## Problem 3: Padding

## Problem 4: Hashes

## Problem 5: Passwords